# AB testing

In [1]:
import sys
sys.path.append("./tools")
from ab_testing import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
AB_test(['f', 't'], [298169.0, 198613.0], [5959.0, 4337.0])

winning: t (significant - confidance: 0.95)
conversion rate A: 1.999%
conversion rate B: 2.184%
uplift: 9.262%
se A: 0.00026
se B: 0.00033
Z-score: 4.44757
p-value: 1e-05
power: 0.99998


In [3]:
pbb_conversion([80000,80000,80000], [1600,1700,1650])

pbb_revenue(
    [4212694,4213358,4333878],
    [639,716,760],
    [2280,2585,2691],
    [8265,9466,9680]
)

[0.0243, 0.7949, 0.1808]

[0.0026, 0.7014, 0.296]